In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt

In [ ]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
# ss = pd.read_csv("../input/tensorflow-great-barrier-reef/example_sample_submission.csv")

train.shape, test.shape

In [ ]:
train['annotations'] = train['annotations'].apply(lambda x: eval(x)) # annotationsが文字列として入っているため、eval()
train['n_annotations'] = train['annotations'].apply(len)

In [ ]:
train.dtypes

In [ ]:
max_annots = 0
max_id = "" 
video_id = 0
for video_frame in range(1001, 1020):
    image_id = f'{video_id}-{video_frame}'
    len_annots = train[train['image_id']==image_id]['n_annotations'].item()
    if len_annots > max_annots:
        max_annots = len_annots
        max_id = image_id

print(max_id)
print(max_annots)

In [ ]:
max_annots = 0
max_id = "" 
for video_id in range(1,4):
    for video_frame in range(10001, 10020):
        image_id = f'{video_id}-{video_frame}'
        len_annots =  train[train['image_id']==image_id]['n_annotations'].item()
        if len_annots > max_annots:
            max_annots = len_annots
            max_id = image_id
                        
print(max_id)
print(max_annots)

- image_id 0-1001か1-10001を使う

In [ ]:
max_id

In [ ]:
max_bboxes = train[train['image_id']==max_id]['annotations'].item()
max_bboxes

- 可視化パクリ元
https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507

In [ ]:
import cv2
from PIL import Image
image_path = '../input/tensorflow-great-barrier-reef/train_images/video_0/1001.jpg'

In [ ]:
def draw_bboxes(img, bboxes, scores, bbclasses, confthre=None, classes_dict=None):
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
#             if score < confthre:
#                 continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = x0 + int(box[2])
            y1 = y0 + int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

In [ ]:
sample_img = cv2.imread(image_path)

# Get predictions
# bboxes, bbclasses, scores = yolox_inference(img, model, test_size)
sample_bboxes = [list(_.values()) for _ in max_bboxes]
bboxes, bbclasses, scores = sample_bboxes, [0], [0.99]

# Draw predictions
# out_image = draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, COCO_CLASSES)
out_image = draw_bboxes(sample_img, bboxes, scores, bbclasses, {0: 'hitode'})

# Since we load image using OpenCV we have to convert it 
out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
display(Image.fromarray(out_image))

# Augmentation
- 参考: https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [ ]:
import albumentations as A

In [ ]:
out_image.shape

In [ ]:
transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', min_area=100, min_visibility=0.1, label_fields=[]))

In [ ]:
transformed = transform(image=sample_img, bboxes=sample_bboxes)
transformed_img = transformed['image']
transformed_bboxes = transformed['bboxes']

In [ ]:
# Since we load image using OpenCV we have to convert it 
bboxes, bbclasses, scores = transformed_bboxes, ["0"], [0.99]
out_image = draw_bboxes(transformed_img, transformed_bboxes, scores, bbclasses)
out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
display(Image.fromarray(out_image))

In [ ]:
transformed_img.shape

In [ ]:
sample_bboxes